In [28]:
import pandas as pd 
import requests
import cloudscraper
import os
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime
import re
import glob


In [7]:
# creating the url 
url ='https://www.myjobmag.com/'
scraper = cloudscraper.create_scraper()
respond = scraper.get(url)
print(respond)
print(respond.status_code)

<Response [200]>
200


In [ ]:
url ='https://www.myjobmag.com/'
scraper = cloudscraper.create_scraper()
respond = scraper.get(url)
soup = BeautifulSoup(respond.text,'html.parser')
soup

In [ ]:
# the head of the website 
soup = BeautifulSoup(respond.text,'html.parser')
print(soup.head)

In [ ]:
url ='https://www.myjobmag.com/featured-jobs'
scraper = cloudscraper.create_scraper()
respond = scraper.get(url)
jobtitle=[]
soup = BeautifulSoup(respond.text,'html.parser')
job_type = soup.find_all("a",style=' ')
for jobname in job_type:
    jobtitle.append(jobname.get_text(strip=True))
jobtitle
    

In [ ]:
# webcraping with beautifulsoup
url ='https://www.myjobmag.com/featured-jobs'
scraper = cloudscraper.create_scraper()
respond = scraper.get(url)
jobdescription =[]
soup = BeautifulSoup(respond.text,'html.parser')
job_description = soup.find_all("li",class_="job-desc")
for i in job_description:
    jobdescription.append(i.get_text(strip=True))
jobdescription
    


In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
from datetime import datetime

url = 'https://www.myjobmag.com/featured-jobs'
scraper = cloudscraper.create_scraper()
response = scraper.get(url)

job_dates = []

soup = BeautifulSoup(response.text, 'html.parser')
job_date_elements = soup.find_all("li", id="job-date")

for element in job_date_elements:
    date_text = element.text.strip() + " 2024"  # Append space before the year
    date_text = datetime.strptime(date_text, '%d %B %Y')
    formatted_date = date_text.strftime('%d-%m-%Y')
    job_dates.append(formatted_date)

job_dates

    

In [ ]:

url = 'https://www.myjobmag.com/featured-jobs'
scraper = cloudscraper.create_scraper()
response = scraper.get(url)

job_role = []

soup = BeautifulSoup(response.text, 'html.parser')
job_section = soup.find_all("li", id="job-info")+ soup.find_all("li",class_="job-list-li")
for section in job_section:
    roles=section.find_all("ul",id="sbu-job-list")
    if roles:
        for role in roles:
            job_role.extend([job.text.strip() for job in role.find_all("a")])
    else:
        job_role.append('no specific description role')
            



job_role

In [ ]:
job_sections = soup.find_all('li', class_='job-info') + soup.find_all('li', class_='job-list-li')
job_urls = []
for section in job_sections:
    h2_tag = section.find("h2")
    if h2_tag:
        a_tag = h2_tag.find("a", href=True)
        if a_tag and 'href' in a_tag.attrs:
            job_urls.append(a_tag['href'])
        else:
            job_urls.append('no link')
job_urls

In [71]:
data = {
    'Job_Title': [],
    'Job_Description': [],
    'Job_Post_Date': [],
    'Job_Specific_Roles': [],
    'Job_Url': []
}
for i in range(len(jobtitle)):
    if i < len(job_role):
        roles = job_role[i].split(',')
        for role in roles:
            data['Job_Title'].append(jobtitle[i])
            data['Job_Description'].append([jobdescription[i] if i<len(jobdescription[i]) else ''])
            data['Job_Post_Date'].append([job_dates[i] if i<len(job_dates[i]) else ''])
            data['Job_Specific_Roles'].append(role.strip())
            data['Job_Url'].append(job_urls[i] if i<len(job_urls[i]) else '')
    else:
        data['job_Title'].append(jobtitle[i])
        data['job_Decription'].append([jobdescription[i] if i<len(jobdescription[i]) else ''])
        data['Job_Post_Date'].append([job_dates[i] if i<len(job_dates[i]) else ''])
        data['Job_Specific_Roles'].append(role.strip())
        data['Job_Url'].append(job_urls[i] if i<len(job_urls[i]) else '')
df =pd.DataFrame(data)
df
            
        

,Job_Title,Job_Description,Job_Post_Date,Job_Specific_Roles,Job_Url
0,Field Coordinator at a Community Healthcare Or...,[This company is looking to recruit qualified ...,[13-07-2024],no specific description role,/job/field-coordinator
1,Technical Assistant to CEO at Water Supply and...,[Water Supply and Sanitation Initiative WASSI ...,[13-07-2024],no specific description role,/job/technical-assistant-to-ceo-water-supply-a...
2,Search Engine Optimization (SEO) at Perfectlytics,[Transform your online presence with our innov...,[13-07-2024],no specific description role,/job/search-engine-optimization-seo-perfectlytics
3,Content Creator/Social Media Manager at Miasa ...,[This company is looking to recruit qualified ...,[13-07-2024],no specific description role,/job/content-creator-social-media-manager
4,Sales Executive at Miasa Homes and Properties ...,[This company is looking to recruit qualified ...,[12-07-2024],no specific description role,/job/sales-executive-82
...,...,...,...,...,...
227,Automotive Technician / Mechanic - Japanese/Ge...,[],[],no specific description role,
228,Fashion House Manager at Pongol Bespoke Nigeri...,[],[],no specific description role,
229,Sales Representative at Gafford Property & Homes,[],[],no specific description role,
230,Security Operations Manager at Amach Security ...,[],[],no specific description role,
